<a href="https://colab.research.google.com/github/rwfchan/ResumeScanner/blob/main/MySQLSearchQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1st parameter: folder path
# 2nd parameter: job title query
# 3rd parameter: degree query
# 4th parameter: major query
# 5th parameter: skill query
resumeFilter("", "", "", "", "")

In [ ]:
# 1st parameter: folder path
# 2nd parameter: partial query call to user's MySQL database
#                "[column_name1] LIKE '%[keyword1]%' AND '%[keyword2]%' AND [column_name2] LIKE '%[keyword3]%'..."
#                where column_namex can be ["Name, Organization, Position, Email, Phone, Skill, SchoolName, SchoolDegree"]

resumeQuery("", "")

In [ ]:
pip install mysql-connector-python

In [ ]:
import mysql.connector
import os

def resumeFilter(path, jobs = None, degrees = None, majors = None , skills = None):
  try:
    # need to delete the database information when move to beta version
    mydb = mysql.connector.connect(host="Enter Host Address", user="Enter Username", passwd="Enter Password", database="Enter Database Name")
    print("Connection to MySQL DB successful")
  except Error as e:
    print(f"The error '{e}' occurred")
  
  mycursor = mydb.cursor()

  job_list = []
  degree = []
  skill_list = []
  major_list = []

  if jobs:
    for job in jobs.split(' '):
      job_list.append(job)
  
  if majors:
    for major in majors.split(' '):
      major_list.append(major)
  
  if degrees:
    degrees = degrees.lower()
    if degrees[0] == 'b':
      degree = ['BS','B S', 'bachelor']
    elif degrees[0] == 'm':
      degree = ['master', 'M S', 'MS']
    elif degrees[0] == 'p':
      degree = ['PhD', 'Ph D', 'Philosophy']
    else:
      print('no such degree')
      degree = []
  
  if skills:
    skill_list = skills.split(',')
  
  # need to change the database and table name
  query = "SELECT * FROM csv_output.resume_scanner_beta WHERE "

  if job_list:
    job_query = "("
    for job in job_list:
      job_query += "Position LIKE '%" + job + "%' AND "
    job_query = job_query[:-5] + ")"
    query += job_query

  if job_list and degree:
    query += " AND "
  
  if degree:
    degree_query = "("
    for degree_title in degree:
      degree_query += "SchoolDegree LIKE '%" + degree_title + "%' OR "
    degree_query = degree_query[:-4] + ")"
    query += degree_query

  if (job_list or degree) and skill_list:
    query += " AND "
  
  if skill_list:
    skill_query = "("
    for skill in skill_list:
      skill_query += "Skill LIKE '%" + skill + "%' AND "
    skill_query = skill_query[:-5] + ")"
    query += skill_query
  
  if (job_list or degree or skill_list) and major_list:
    query += " AND "
  
  if major_list:
    major_query = "("
    for major in major_list:
      major_query += "SchoolDegree LIKE '%" + major + "%' AND "
    major_query = major_query[:-5] + ")"
    query += major_query
  
  import pandas as pd
  print("Reading BLOB data from resume_scanner_beta table")

  try:
        
      mycursor.execute(query)
      record = mycursor.fetchall()
      print(record)

      directory = 'output'
      dir_path = os.path.join(path,directory)

      if not os.path.isdir(dir_path):
        os.mkdir(dir_path)

      for row in record:
          image = row[9]
          print("Storing employee image and bio-data on disk \n")
          new_path = dir_path + "/" + row[0][2:-2] + ".jpg"
          write_file(image, new_path)

  except mysql.connector.Error as error:
      print("Failed to read BLOB data from MySQL table {}".format(error))

  finally:
      if (mydb.is_connected()):
          mycursor.close()
          mydb.close()
          print("MySQL connection is closed")

In [ ]:
# SQL injection
# input: folder path, sql query
# output: the jpg file
import mysql.connector
import os

def resumeQuery(path, user_query):
  try:
    # need to delete the database information when move to beta version
    mydb = mysql.connector.connect(host="Enter Host Address", user="Enter Username", passwd="Enter Password", database="Enter Database Name")
    print("Connection to MySQL DB successful")
  except Error as e:
    print(f"The error '{e}' occurred")
  
  mycursor = mydb.cursor()
  
  # need to change the database and table name
  query = "SELECT * FROM csv_output.resume_scanner_beta WHERE "

  final_query = query + user_query

  import pandas as pd
  print("Reading BLOB data from resume_scanner_beta table")

  try:
        
      mycursor.execute(final_query)
      record = mycursor.fetchall()
      print(record)

      directory = 'output'
      dir_path = os.path.join(path,directory)

      if not os.path.isdir(dir_path):
        os.mkdir(dir_path)

      for row in record:
          image = row[9]
          print("Storing employee image and bio-data on disk \n")
          new_path = dir_path + "/" + row[0][2:-2] + ".jpg"
          write_file(image, new_path)

  except mysql.connector.Error as error:
      print("Failed to read BLOB data from MySQL table {}".format(error))

  finally:
      if (mydb.is_connected()):
          mycursor.close()
          mydb.close()
          print("MySQL connection is closed")

In [ ]:
def write_file(data, filename):
    # Convert binary data to proper format and write it on Hard Disk
    with open(filename, 'wb') as file:
        file.write(data)